In [ ]:
import shutil

In [ ]:
CLASS_NAMES = ['표고']

In [ ]:
# object_name과 원본 좌표를 입력하여 Yolo 포맷으로 변환
def convert_yolo_coord(object_name, img_width, img_height, x, y, w, h):
  class_id = CLASS_NAMES.index(object_name)
  center_x_norm = (x + w/2) / img_width
  center_y_norm = (y + h/2) / img_height
  width_norm = w / img_height
  height_norm = h / img_height

  return class_id, round(center_x_norm, 7), round(center_y_norm, 7), round(width_norm, 7), round(height_norm, 7)

In [ ]:
# 1개의 annotation json 파일을 Yolo 포맷용 txt 파일로 변경하는 함수
def json_to_txt(input_json_file, output_txt_file, object_name):
  # Load json file
  with open(input_json_file) as json_file:
    anno_data = json.load(json_file)

  # Width and height of input image
  img_data = anno_data['IMAGE']
  img_width = img_data['WIDTH']
  img_height = img_data['HEIGHT']

  if img_data['ANNOTATION_COUNT'] == 0:
    return None

  value_str = None
  # json 파일 내에 있는 모든 object에 대해서
  with open(output_txt_file, 'w') as output_fpointer:
    for obj in anno_data['ANNOTATION_INFO']:
      # Bounding box coordinate (x, y), width, height
      x = obj['BOUNDING_BOX_X_COORDINATE']
      y = obj['BOUNDING_BOX_Y_COORDINATE']
      w = obj['BOUNDING_BOX_WIDTH']
      h = obj['BOUNDING_BOX_HEIGHT']

      # Yolo annotation 포맷으로 변환
      class_id, cx_norm, cy_norm, w_norm, h_norm = convert_yolo_coord(object_name, img_width, img_height, x, y, w, h)
      value_str = ('{0} {1} {2} {3} {4}').format(class_id, cx_norm, cy_norm, w_norm, h_norm)
      output_fpointer.write(value_str+'\n')

In [ ]:
def make_yolo_anno_file(df, tgt_images_dir, tgt_labels_dir):
  for index, row in df.iterrows():
    src_image_path = row['img_filepath']
    src_label_path = row['anno_filepath']
    object_name = row['class_name']
    # image의 경우 target images 디렉토리로 단순 copy
    shutil.copy(src_image_path, tgt_images_dir)
    # yolo format으로 annotation할 txt 파일의 절대 경로명을 지정
    target_label_path = tgt_labels_dir + row['filename'] + '.txt'
    # annotation의 경우 xml 파일을 target labels 디렉토리에 Ultralytics Yolo format으로 변환하여  만듬
    json_to_txt(src_label_path, target_label_path, object_name)

In [ ]:
# train용 images와 labels annotation 생성.
# make_yolo_anno_file(train_tiny_df, '/kaggle/working/dataset/images/train/', '/kaggle/working/dataset/labels/train/')

# val용 images와 labels annotation 생성
# make_yolo_anno_file(val_tiny_df, '/kaggle/working/dataset/images/val/', '/kaggle/working/dataset/labels/val/')